In [1]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
import pickle

In [2]:
# Load dataset
features = np.load('features.npy')
labels = np.load('target.npy')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Base models
base_models = [
    ('rf', make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100, random_state=42))),
    ('gb', make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=42))),
    ('svc', make_pipeline(StandardScaler(), SVC(probability=True, random_state=42)))
]

# Meta-model
meta_model = LogisticRegression()

# Stacking Classifier
stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Train the stacking classifier
stacking_clf.fit(X_train, y_train)

# Predict the test set results
y_pred = stacking_clf.predict(X_test)

# Evaluate the results
print(classification_report(y_test, y_pred))
print('Train Score:', stacking_clf.score(X_train, y_train))
print('Test Score:', stacking_clf.score(X_test, y_test))

# Save the best model using pickle
with open('stacking_model.pkl', 'wb') as f:
    pickle.dump(stacking_clf, f)

              precision    recall  f1-score   support

           0       0.85      0.83      0.84        86
           1       0.81      0.88      0.84        66
           2       0.97      0.99      0.98        73
           3       0.83      0.77      0.80        84
           4       0.92      0.95      0.93        59
           5       0.75      0.87      0.80        61
           6       0.95      0.82      0.88        67
           7       0.90      0.88      0.89        74

    accuracy                           0.87       570
   macro avg       0.87      0.87      0.87       570
weighted avg       0.87      0.87      0.87       570

Train Score: 1.0
Test Score: 0.868421052631579
